In [ ]:
#get Thirukkural text data from kaggle dataset
from google.colab import drive
drive.mount('/content/gdrive')
%mkdir /content/kaggle
%cp "/content/gdrive/My Drive/kaggle/kaggle.json" "/content/kaggle/kaggle.json"

import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/kaggle"
%cd /content/kaggle

!kaggle datasets download -d neechalkaran/thirukkural
!unzip \*.zip  && rm *.zip


import pandas as pd
  
# Then loading csv file
df = pd.read_csv('/content/kaggle/Thirukkural.csv')

  
a = list(df['Yogi Shuddhananda Commentary'])
b = list(df['G.U. Pope, Rev. Drew and John Lazarus Translation'])
c = list(df['G.U. Pope, Rev. Drew and John Lazarus Commentary'])
d = list(df['Satguru Sivaya Subramuniyaswami.1'])

data = '\n'.join(str(e) for e in a) +'\n'+ '\n'.join(str(e) for e in b) +'\n'+ '\n'.join(str(e) for e in c) +'\n'+ '\n'.join(str(e) for e in d)

file = open("/content/corpus.txt", 'a') # write
file.write(data)
file.close()  

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
mkdir: cannot create directory ‘/content/kaggle’: File exists
/content/kaggle
 84% 1.00M/1.19M [00:00<00:00, 2.00MB/s]
100% 1.19M/1.19M [00:00<00:00, 2.30MB/s]
Archive:  thirukkural.zip
replace Thirukkural.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Thirukkural.csv         


In [ ]:
!pip install openai
!pip install langchain==0.0.107
!pip install gpt_index==0.4.24
import openai
OPENAI_API_KEY="<---put your key---->"
Org="<------put your Org-------->"
openai.organization = Org
openai.api_key = OPENAI_API_KEY 
#https://beebom.com/how-train-ai-chatbot-custom-knowledge-base-chatgpt-api/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Training the custom data with existing GPT Model
from gpt_index import SimpleDirectoryReader, GPTListIndex, GPTSimpleVectorIndex, LLMPredictor, PromptHelper
from langchain.chat_models import ChatOpenAI

import sys
import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

def construct_index(directory_path):
    max_input_size = 4096
    num_outputs = 512
    max_chunk_overlap = 20
    chunk_size_limit = 600

    prompt_helper = PromptHelper(max_input_size, num_outputs, max_chunk_overlap, chunk_size_limit=chunk_size_limit)

    llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0.7, model_name="gpt-3.5-turbo", max_tokens=num_outputs))

    documents = SimpleDirectoryReader(directory_path).load_data()

    index = GPTSimpleVectorIndex(documents, llm_predictor=llm_predictor, prompt_helper=prompt_helper)

    index.save_to_disk('/content/index.json')

    return index

def chatbot(input_text):
    index = GPTSimpleVectorIndex.load_from_disk('/content/index.json')
    response = index.query(input_text, response_mode="compact")
    return response.response



index = construct_index("/content/kural")


In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
print(chatbot("share 2 beautiful romatic lines of thirukkural"))


1. "The love between two hearts is like the bond between the sun and the moon, never to be broken." 
2. "The love between two hearts is like the bond between the body and the soul, never to be parted."


In [ ]:
print(chatbot("what are the principles to earn money?"))


1. Ponder the potential risks and rewards before taking action.
2. Consult with trusted advisors before making decisions.
3. Avoid taking on projects that are unclear or uncertain.
4. Have a well-thought-out plan before beginning a task.
5. Consider the strengths and weaknesses of yourself and your opponents.
6. Understand the nature and means of the task before attempting it.
7. Know your own capacity and act accordingly.
8. Spend within your means.
9. Measure your wealth and resources carefully.
10. Choose the right time to act.


In [ ]:
print(chatbot("what are the duties of the children?"))


The duties of the children are to make their parents proud by striving to be knowledgeable and wise, to show love and affection to their parents and others, to be hospitable to guests, and to be virtuous and loving in their actions.


In [ ]:
print(chatbot("tell about why virtue is important?"))


Virtue is important because it yields Heaven's honor and Earth's wealth, and is more rewarding than any other pursuit. It is the practice of living in such a way that one does not fall into envy, anger, greed, or unsavory speech. Virtue is also a deathless companion when one dies, and is the only way to ensure true joy. Doing good deeds every day is essential to living a virtuous life, and is the only way to block the path to rebirth.
